## Semi-auto Stream in Landsat from USGS
Solution: Use brower-cookie3 to "sign" the URL request so a python script can stream data from USGS
Cons: Low bandwidth ~200KB/s 

In [ ]:
#%pip install browser-cookie3

In [75]:
from pathlib import Path
import browser_cookie3
import requests
import shutil
import os
import json
from urllib.request import urlopen
from pystac.item import Item
import pandas as pd
from dateutil.rrule import rrule, DAILY
import datetime
from datetime import date
import glob

In [19]:
#official script for m2m api

import json
import requests
import sys
import time
import argparse
import re
import threading
import datetime
import os
from dotenv import load_dotenv

load_dotenv()


path = "../../data/landsat/m2m_download" # Fill a valid download path
maxthreads = 10 # Threads count for downloads
sema = threading.Semaphore(value=maxthreads)
label = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") # Customized label using date time
threads = []
# The entityIds/displayIds need to save to a text file such as scenes.txt.
# The header of text file should follow the format: datasetName|displayId or datasetName|entityId. 
# sample file - scenes.txt
# landsat_ot_c2_l2|displayId
# LC08_L2SP_012025_20201231_20210308_02_T1
# LC08_L2SP_012027_20201215_20210314_02_T1
scenesFile = '../pyScripts/scenes2.txt'

# Send http request
def sendRequest(url, data, apiKey = None, exitIfNoResponse = True):  
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}              
        response = requests.post(url, json_data, headers = headers)  
    
    try:
      httpStatusCode = response.status_code 
      if response == None:
          print("No output from service")
          if exitIfNoResponse: sys.exit()
          else: return False
      output = json.loads(response.text)
      if output['errorCode'] != None:
          print(output['errorCode'], "- ", output['errorMessage'])
          if exitIfNoResponse: sys.exit()
          else: return False
      if  httpStatusCode == 404:
          print("404 Not Found")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 401: 
          print("401 Unauthorized")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 400:
          print("Error Code", httpStatusCode)
          if exitIfNoResponse: sys.exit()
          else: return False
    except Exception as e: 
          response.close()
          print(e)
          if exitIfNoResponse: sys.exit()
          else: return False
    response.close()
    
    return output['data']

def downloadFile(url):
    sema.acquire()
    try:        
        response = requests.get(url, stream=True)
        disposition = response.headers['content-disposition']
        filename = re.findall("filename=(.+)", disposition)[0].strip("\"")
        print(f"Downloading {filename} ...\n")
        if path != "" and path[-1] != "/":
            filename = "/" + filename
        open(path+filename, 'wb').write(response.content)
        print(f"Downloaded {filename}\n")
        sema.release()
    except Exception as e:
        print(f"Failed to download from {url}. Will try to re-download.")
        sema.release()
        runDownload(threads, url)
    
def runDownload(threads, url):
    thread = threading.Thread(target=downloadFile, args=(url,))
    threads.append(thread)
    thread.start()

In [149]:
username = os.getenv("M2M_USER")
password = os.getenv("M2M_PSWD")  

print("\nRunning Scripts...\n")

serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"

# login
payload = {'username' : username, 'password' : password}

apiKey = sendRequest(serviceUrl + "login", payload)

print("API Key: " + apiKey + "\n")

datasetName = "landsat_ot_c2_l1"

spatialFilter =  {'filterType' : "mbr",
                  'lowerLeft' : {'latitude' : 30, 'longitude' : -120},
                  'upperRight' : { 'latitude' : 40, 'longitude' : -140}}

temporalFilter = {'start' : '2021-10-10', 'end' : '2025-12-10'}

payload = {'datasetName' : datasetName,
                           'temporalFilter' : temporalFilter}                     

print("Searching datasets...\n")
datasets = sendRequest(serviceUrl + "dataset-search", payload, apiKey)

print("Found ", len(datasets), " datasets\n")


Running Scripts...

API Key: eyJjaWQiOjI2NDc1ODQwLCJzIjoiMTY2ODA1NDkxNyIsInIiOjk4NywicCI6WyJ1c2VyIiwiZG93bmxvYWQiLCJvcmRlciJdfQ==

Searching datasets...

Found  1  datasets



In [173]:
acquisitionFilter = {"end": "2021-11-30",
                             "start": "2021-10-10" }        
metadataValue = {"filterType":"value",
                  "filterId":"61af93b8fad2acf5",
                  "value":"9",
                   "operand":"="
                 }


payload = {'datasetName' : 'landsat_ot_c2_l1', 
                         'maxResults' : 50000,
                         #'startingNumber' : 1, 
                         'sceneFilter' : {'metadataFilter':metadataValue,
                                          #'spatialFilter' : spatialFilter,
                                          'acquisitionFilter' : acquisitionFilter}}

# Now I need to run a scene search to find data to download
print("Searching scenes...\n\n")   

scenes = sendRequest(serviceUrl + "scene-search", payload, apiKey)

Searching scenes...




In [174]:
len(scenes['results'])

12215

In [175]:
scenes['results'][400]['displayId']

'LC09_L1TP_168055_20211124_20220120_02_T1'

In [20]:
username = os.getenv("M2M_USER")
password = os.getenv("M2M_PSWD")
filetype = 'bundle'

print("\nRunning Scripts...\n")
startTime = time.time()

serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"

# Login
payload = {'username' : username, 'password' : password}    
apiKey = sendRequest(serviceUrl + "login", payload)    
print("API Key: " + apiKey + "\n")

# Read scenes
f = open(scenesFile, "r")
lines = f.readlines()   
f.close()
header = lines[0].strip()
datasetName = header[:header.find("|")]
idField = header[header.find("|")+1:]

print("Scenes details:")
print(f"Dataset name: {datasetName}")
print(f"Id field: {idField}\n")

entityIds = []

lines.pop(0)
for line in lines:        
    entityIds.append(line.strip())

# Search scenes 
# If you don't have a scenes text file that you can use scene-search to identify scenes you're interested in
# https://m2m.cr.usgs.gov/api/docs/reference/#scene-search
# payload = { 
#             'datasetName' : '', # dataset alias
#             'maxResults' : 10, # max results to return
#             'startingNumber' : 1, 
#             'sceneFilter' : {} # scene filter
#           }

# results = sendRequest(serviceUrl + "scene-search", payload, apiKey)  
# for result in results:
#     entityIds.append(result['entityId'])

# Add scenes to a list
listId = f"temp_{datasetName}_list" # customized list id
payload = {
    "listId": listId,
    'idField' : idField,
    "entityIds": entityIds,
    "datasetName": datasetName
}

print("Adding scenes to list...\n")
count = sendRequest(serviceUrl + "scene-list-add", payload, apiKey)    
print("Added", count, "scenes\n")

# Get download options
payload = {
    "listId": listId,
    "datasetName": datasetName
}

print("Getting product download options...\n")
products = sendRequest(serviceUrl + "download-options", payload, apiKey)
print("Got product download options\n")

# Select products
downloads = []
if filetype == 'bundle':
    # select bundle files
    for product in products:        
        if product["bulkAvailable"]:               
            downloads.append({"entityId":product["entityId"], "productId":product["id"]})
elif filetype == 'band':
    # select band files
    for product in products:  
        if product["secondaryDownloads"] is not None and len(product["secondaryDownloads"]) > 0:
            for secondaryDownload in product["secondaryDownloads"]:
                if secondaryDownload["bulkAvailable"]:
                    downloads.append({"entityId":secondaryDownload["entityId"], "productId":secondaryDownload["id"]})
else:
    # select all available files
    for product in products:        
        if product["bulkAvailable"]:               
            downloads.append({"entityId":product["entityId"], "productId":product["id"]})
            if product["secondaryDownloads"] is not None and len(product["secondaryDownloads"]) > 0:
                for secondaryDownload in product["secondaryDownloads"]:
                    if secondaryDownload["bulkAvailable"]:
                        downloads.append({"entityId":secondaryDownload["entityId"], "productId":secondaryDownload["id"]})

# Remove the list
payload = {
    "listId": listId
}
sendRequest(serviceUrl + "scene-list-remove", payload, apiKey)                

# Send download-request
payLoad = {
    "downloads": downloads,
    "label": label,
    'returnAvailable': True
}

print(f"Sending download request ...\n")
results = sendRequest(serviceUrl + "download-request", payLoad, apiKey)
print(f"Done sending download request\n") 


Running Scripts...

API Key: eyJjaWQiOjI2NDc1ODQwLCJzIjoiMTY2NzY1NzMzNiIsInIiOjM3NSwicCI6WyJ1c2VyIiwiZG93bmxvYWQiLCJvcmRlciJdfQ==

Scenes details:
Dataset name: landsat_ot_c2_l1
Id field: displayId

Adding scenes to list...

Added 200 scenes

Getting product download options...

Got product download options

Sending download request ...

Done sending download request



In [ ]:
results

In [26]:
results['availableDownloads'][1]['url']

'https://landsatlook.usgs.gov/gen-browse?size=frb&type=refl&product_id=LC09_L1GT_002051_20211122_20220120_02_T2&requestSignature=eyJjb250YWN0SWQiOjI2NDc1ODQwLCJkb3dubG9hZElkIjozMjc5MjI0NjksImRhdGVHZW5lcmF0ZWQiOiIyMDIyLTExLTA1VDA5OjA5OjI2LTA1OjAwIiwic2lnbmF0dXJlIjoiJDUkJFwvR3pBQUFxb2VoVENwYnBRMU9uZjZNZWlVY2dNS1QwR2VtXC9WQ0tURkVlMCJ9'

In [27]:
from urllib.parse import urlparse

In [ ]:
for result in results['availableDownloads']:
    u = urlparse(result['url'])
    if u.path.split('/')[-1]!='gen-browse':
        

'gen-browse'

In [ ]:
for result in results['availableDownloads']:       
    print(f"Get download url: {result['url']}\n" )
    runDownload(threads, result['url'])

preparingDownloadCount = len(results['preparingDownloads'])
preparingDownloadIds = []
if preparingDownloadCount > 0:
    for result in results['preparingDownloads']:  
        preparingDownloadIds.append(result['downloadId'])

    payload = {"label" : label}                
    # Retrieve download urls
    print("Retrieving download urls...\n")
    results = sendRequest(serviceUrl + "download-retrieve", payload, apiKey, False)
    if results != False:
        for result in results['available']:
            if result['downloadId'] in preparingDownloadIds:
                preparingDownloadIds.remove(result['downloadId'])
                print(f"Get download url: {result['url']}\n" )
                runDownload(threads, result['url'])

        for result in results['requested']:   
            if result['downloadId'] in preparingDownloadIds:
                preparingDownloadIds.remove(result['downloadId'])
                print(f"Get download url: {result['url']}\n" )
                runDownload(threads, result['url'])

    # Don't get all download urls, retrieve again after 30 seconds
    while len(preparingDownloadIds) > 0: 
        print(f"{len(preparingDownloadIds)} downloads are not available yet. Waiting for 30s to retrieve again\n")
        time.sleep(30)
        results = sendRequest(serviceUrl + "download-retrieve", payload, apiKey, False)
        if results != False:
            for result in results['available']:                            
                if result['downloadId'] in preparingDownloadIds:
                    preparingDownloadIds.remove(result['downloadId'])
                    print(f"Get download url: {result['url']}\n" )
                    runDownload(threads, result['url'])

print("\nGot download urls for all downloads\n")                
# Logout
endpoint = "logout"  
if sendRequest(serviceUrl + endpoint, None, apiKey) == None:        
    print("Logged Out\n")
else:
    print("Logout Failed\n")  

print("Downloading files... Please do not close the program\n")
for thread in threads:
    thread.join()

print("Complete Downloading")

executionTime = round((time.time() - startTime), 2)
print(f'Total time: {executionTime} seconds')

In [2]:
# background step: login in USGS EROS so the brower cookie can skip the redirect of USGS when request

cj = browser_cookie3.firefox(domain_name='usgs.gov')

In [3]:
# single file

url = "https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2020/047/027/LC08_L2SP_047027_20201204_20210313_02_T1/LC08_L2SP_047027_20201204_20210313_02_T1_SR_B7.TIF"
url = "https://landsatlook.usgs.gov/data/collection02/level-1/standard/oli-tirs/2022/008/009/LC09_L1GT_008009_20221014_20221014_02_T2/LC09_L1GT_008009_20221014_20221014_02_T2_B2.TIF"

url
r = requests.get(url, stream = True,cookies=cj)


fname = Path(url).name
if r.status_code == 200:
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    r.raw.decode_content = True
    
    # Open a local file with wb ( write binary ) permission.
    with open(fname,'wb') as f:
        shutil.copyfileobj(r.raw, f)

In [4]:
def cache_asset(url,path):
    r = requests.get(url, stream = True,cookies=cj)
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True
        
        # Open a local file with wb ( write binary ) permission.
        with open(path,'wb') as f:
            shutil.copyfileobj(r.raw, f)

In [ ]:
target

In [15]:
usgs_item_url = f"https://landsatlook.usgs.gov/stac-server/collections/landsat-c2l1/items/LC09_L1TP_094083_20211110_20220119_02_T1"
#read stac
r = urlopen(usgs_item_url).read()
stac_json = json.loads(r)

In [23]:

def hand_pull_product(product_id,collection="landsat-c2l1",tarfile=True,verbose=False):
    """
    product_id: e.g."LC09_L1TP_086075_20220509_20220510_02_T1"
    """
    directory = f"../../data/landsat/hand_pull/{product_id}"
    if os.path.exists(directory+".tar"):
        #print('skip',product_id)
        return
    usgs_item_url = f"https://landsatlook.usgs.gov/stac-server/collections/{collection}/items/{product_id}"
    #aws_item_url = "https://jcz3phgts3.execute-api.us-west-2.amazonaws.com/dev/getobject/collection02/level-1/standard/oli-tirs/2021/094/083/LC09_L1TP_094083_20211110_20220119_02_T1/LC09_L1TP_094083_20211110_20220119_02_T1_stac.json"
    #read stac
    r = urlopen(usgs_item_url).read()
    stac_json = json.loads(r)
    scene = Item.from_dict(stac_json)
    assets_urls = [[k,scene.assets[k].href] for k in scene.get_assets().keys() if k != 'index']
    directory = f"../../data/landsat/hand_pull/{scene.id}" #in case there is a miss match
    if not os.path.exists(directory):
        os.makedirs(directory)
    #loop run the assets
    for k,url in assets_urls:
        path = f"{directory}/{Path(url).name}"
        #print(k,path)
        if verbose:
            print(url)
        cache_asset(url,path)
    # save stac information
    with open(f"{directory}/{scene.id}_stac.json","w") as w:
        json.dump(stac_json,w)
    if tarfile:
        output = shutil.make_archive(directory, 'tar', Path(directory).parent, Path(directory).name)
        print(output)
        shutil.rmtree(directory)
    else:
        print(directory)

In [7]:
missing_products_l1 = pd.read_csv("../../../data/Landsat/missing_id.csv").id.tolist()

In [7]:
chronological_list = sorted(missing_products_l1, key=lambda x: x.split('_')[3])

In [8]:
len(chronological_list)

128879

In [2]:
import glob

In [67]:
#debugging single file
#prd_id = finished[0]
def move_to_row_path(prd_id,sub_folder):
    row,path = prd_id.split('_')[2][0:3],prd_id.split('_')[2][3:]
    dst = f"../../data/landsat/{row}/{path}/"
    if not os.path.exists(dst):
        os.makedirs(dst)
    shutil.move(f"../../data/landsat/{sub_folder}/{prd_id}.tar",dst)

In [260]:
finished_m2m= [fp.split('/')[-1] for fp in glob.glob("../../data/landsat/m2m_download/*/*")]

In [227]:
# for asset in finished_folder:
#     if len(os.listdir(asset))!= 24:
#         print(asset,len(os.listdir(asset)))

In [97]:
# archive
finished_handpull = [fp.split('/')[-1][:-4] for fp in glob.glob("../../data/landsat/m2m_download/*.tar")]
import tqdm
for prd_id in tqdm.tqdm(finished_handpull):
    move_to_row_path(prd_id,"m2m_download")

100%|███████████████████████████████████████| 200/200 [00:02<00:00, 98.14it/s]


In [98]:
finished = [fp.split('/')[-1][:-4] for fp in glob.glob("../../data/landsat/**/*.tar",recursive=True)]

In [99]:
len(finished)

136637

In [40]:
#hand_pull_product("LO09_L1TP_153041_20220312_20220315_02_T1")

In [41]:
chronological_list = sorted(list(set(missing_products_l1)-set(finished)), key=lambda x: x.split('_')[3])

In [42]:
len(chronological_list)

66089

In [63]:
chronological_list[19100]

'LC09_L1TP_014014_20220501_20220501_02_T1'

In [94]:
len(todo)

200

In [127]:
len(m2m_batch1) // 100 +1

123

In [195]:
def export_scenes_list(many_scenes,batch_tag,batch_size=100):
    batch_num = len(many_scenes) // batch_size +1
    if len(many_scenes) % batch_size == 0:
        batch_num-=1
    import os
    dest = f'../../data/landsat/task/{batch_tag}'
    try:
        os.makedirs(dest)
    except FileExistsError:
       # directory already exists
       pass
    for i in range(1,batch_num+1):
        with open(f'{dest}/scenes_{str(i).zfill(4)}.txt', 'w') as fp:
            fp.write("landsat_ot_c2_l1|displayId\n")
            for item in many_scenes[(i-1)*batch_size:i*batch_size]:
                # write each item on a new line
                fp.write(f"{item}\n")
    print('Done',batch_num)

In [181]:
export_scenes_list(m2m_batch2,200)

Done 97


In [96]:
with open(r'../pyScripts/scenes.txt', 'w') as fp:
    fp.write("landsat_ot_c2_l1|displayId\n")
    for item in todo:
        # write each item on a new line
        fp.write(f"{item}\n")
    print('Done')

Done


In [52]:
!find ../pyScripts/ -type f -name scenes*.txt

../pyScripts/scenes_21.txt
../pyScripts/scenes_68.txt
../pyScripts/scenes_59.txt
../pyScripts/scenes_60.txt
../pyScripts/scenes_27.txt
../pyScripts/scenes_51.txt
../pyScripts/scenes_65.txt
../pyScripts/scenes_7.txt
../pyScripts/scenes_63.txt
../pyScripts/scenes_34.txt
../pyScripts/scenes_49.txt
../pyScripts/scenes_33.txt
../pyScripts/scenes_71.txt
../pyScripts/scenes_46.txt
../pyScripts/scenes_47.txt
../pyScripts/scenes_64.txt
../pyScripts/scenes_2.txt
../pyScripts/scenes_3.txt
../pyScripts/scenes_15.txt
../pyScripts/scenes_32.txt
../pyScripts/scenes_13.txt
../pyScripts/scenes_42.txt
../pyScripts/scenes_66.txt
../pyScripts/scenes_9.txt
../pyScripts/scenes_52.txt
../pyScripts/scenes_6.txt
../pyScripts/scenes_26.txt
../pyScripts/scenes_8.txt
../pyScripts/scenes_70.txt
../pyScripts/scenes_19.txt
../pyScripts/scenes_36.txt
../pyScripts/scenes_11.txt
../pyScripts/scenes_22.txt
../pyScripts/scenes_35.txt
../pyScripts/scenes_74.txt
../pyScripts/scenes_4.txt
../pyScripts/scenes_17.txt
../pyScr

In [12]:
chronological_list[22000]

'LC09_L1TP_126007_20220331_20220331_02_T1'

In [10]:
chronological_list[40000]

'LC09_L1TP_110012_20220518_20220518_02_T1'

In [ ]:
#stress test
start = 9
for it in range(start,start+10)
    prd_id = missing_products_l1[it]
    
    print(datetime.datetime.now(),prd_id)
    hand_pull_product(prd_id,collection="landsat-c2l1")

In [25]:
def dry_run(product_id,collection):
    #print(product_id,collection)
    import time
    time.s

In [ ]:
zip(a_args, repeat(second_arg)

In [ ]:
#stress test multiple threading
from multiprocessing import Pool
from itertools import repeat
print(datetime.datetime.now())
start = 39
a_args = missing_products_l1[start:start+961]
second_arg = "landsat-c2l1"
with Pool(10) as p:
    M = p.starmap(hand_pull_product, zip(a_args, repeat(second_arg)))
print(datetime.datetime.now())


In [32]:
import tqdm

In [ ]:
from multiprocessing import Pool
from itertools import repeat
a_args = chronological_list
second_arg = "landsat-c2l1"
with Pool(10) as p:
    M = p.starmap(dry_run, tqdm.tqdm(zip(a_args, repeat(second_arg)),total=len(a_args)))

In [ ]:
#stress test robustness # preferred deployed
from multiprocessing import Pool
from itertools import repeat
print(datetime.datetime.now())
a_args = chronological_list
second_arg = "landsat-c2l1"
with Pool(10) as p:
    M = p.starmap(hand_pull_product, zip(a_args, repeat(second_arg)))
print(datetime.datetime.now())


## collect inventory from USGS
Query guide: https://landsatlook.usgs.gov/stac-server/api.html#tag/Item-Search


Example URL:
https://landsatlook.usgs.gov/stac-server/collections/landsat-c2l1/items?limit=10000&datetime=2022-01-06T00:00:00Z/2022-01-07T00:00:00Z&fields=id,-type,-geometry,-bbox,-properties,-links,-assets,-collection,-features

In [3]:


a = date(2022, 9, 30)
b = date(2022, 10, 8)

for dt in rrule(DAILY, dtstart=a, until=b):
    print(dt.strftime("%Y-%m-%d"))
    yymmdd = dt.strftime("%Y-%m-%d")
    dt

2022-09-30
2022-10-01
2022-10-02
2022-10-03
2022-10-04
2022-10-05
2022-10-06
2022-10-07
2022-10-08


In [19]:
cltns = ["landsat-c2l2-sr",	# Landsat Collection 2 Level-2 UTM Surface Reflectance (SR) Product
"landsat-c2l2-st",	# Landsat Collection 2 Level-2 UTM Surface Temperature (ST) Product
"landsat-c2ard-st",	# Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Surface Temperature (ST) Product
"landsat-c2l2alb-bt",	# Landsat Collection 2 Level-2 Albers Top of Atmosphere Brightness Temperature (BT) Product
"landsat-c2l3-fsca",	# Landsat Collection 2 Level-3 Fractional Snow Covered Area (fSCA) Product
"landsat-c2ard-bt",	# Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Top of Atmosphere Brightness Temperature (BT) Product
"landsat-c2l1",	# Landsat Collection 2 Level-1 Product
"landsat-c2l3-ba",	# Landsat Collection 2 Level-3 Burned Area (BA) Product
"landsat-c2l2alb-st",	# Landsat Collection 2 Level-2 Albers Surface Temperature (ST) Product
"landsat-c2ard-sr",	# Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Surface Reflectance (SR) Product
"landsat-c2l2alb-sr",	# Landsat Collection 2 Level-2 Albers Surface Reflectance (SR) Product
"landsat-c2l2alb-ta",	# Landsat Collection 2 Level-2 Albers Top of Atmosphere (TA) Reflectance Product
"landsat-c2l3-dswe",	 #Landsat Collection 2 Level-3 Dynamic Surface Water Extent (DSWE) Product
"landsat-c2ard-ta"]    #	Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Top of Atmosphere (TA) Reflectance Product

In [20]:
import os


for cltn in cltns:
    directory = f"../../data/landsat/stac/{cltn}"
    if not os.path.exists(directory):
        os.makedirs(directory)


In [27]:
cltn = "landsat-c2l3-fsca"

In [28]:
url = f'''https://landsatlook.usgs.gov/stac-server/collections/{cltn}/items?limit=10000&datetime={yymmdd}T00:00:00Z/{yymmdd}T23:59:59Z&fields=id,-type,-geometry,-bbox,properties,-links,-assets,-collection,-features'''

In [29]:
stat_list = []

In [34]:

    print('has')

has


In [ ]:

with urlopen(url) as uo:
    data = json.load(uo)
    #print(data)
    if not data['features']:
    feature_df = pd.json_normalize(data['features'])
    l9 = feature_df.query('`properties.platform`=="LANDSAT_9"')
    l8 = feature_df.query('`properties.platform`=="LANDSAT_8"')
    stat_list.append({'date':yymmdd,'total':data['numberMatched'],'return':data['numberReturned'],'LC09':len(l9),'LC08':len(l9)})
    feature_df.drop('type',axis=1).to_csv(f'../../data/landsat/stac/{cltn}_{dt.strftime("%Y-%m-%d")}.csv',index=False)


In [ ]:
pd.DataFrame(stat_list).query('`date` > "2021-09-27"').to_csv('../../data/landsat/day_sum_LC09_LC08.csv',index=False)

In [ ]:
day_sum = pd.DataFrame(stat_list).query('`date` > "2021-09-27"')

In [ ]:
pd.json_normalize(data['features']).query('`properties.platform`=="LANDSAT_9"')

### Batch processing

In [17]:
cltn

'landsat-c2l2-sr'

In [72]:
def inventory_collection(cltn):
    a = date(2021, 10, 15)
    b = date(2022, 5, 1)
    stat_list=[]
    for dt in rrule(DAILY, dtstart=a, until=b):
        #print(dt.strftime("%Y-%m-%d"))
        yymmdd = dt.strftime("%Y-%m-%d")
        url = f'''https://landsatlook.usgs.gov/stac-server/collections/{cltn}/items?limit=10000&datetime={yymmdd}T00:00:00Z/{yymmdd}T23:59:59Z&fields=id,-type,-geometry,-bbox,properties,-links,-assets,-collection,-features'''
        try:
            with urlopen(url) as uo:
                data = json.load(uo)
                #print(data)
                if not data['features']:
                    continue
                feature_df = pd.json_normalize(data['features'])
                l9 = feature_df.query('`properties.platform`=="LANDSAT_9"')
                l8 = feature_df.query('`properties.platform`=="LANDSAT_8"')
                stat_list.append({'date':yymmdd,'total':data['numberMatched'],'return':data['numberReturned'],'LC09':len(l9),'LC08':len(l8)})
                if len(l9)>0:
                    l9.drop('type',axis=1).to_csv(f'../../data/landsat/stac/{cltn}/{cltn}_{dt.strftime("%Y-%m-%d")}_L9.csv',index=False)
                #print(stat_list[-1])
        except e:
            print(e)
    pd.DataFrame(stat_list).to_csv(f'../../data/landsat/stac/stat/{cltn}_day_sum_LC09_LC08.csv',index=False)


In [76]:
inventory_collection("landsat-c2l1")

In [36]:
for cltn in cltns[4:-1]:
    print(cltn)
    inventory_collection(cltn)

landsat-c2l3-fsca
landsat-c2ard-bt
landsat-c2l1
landsat-c2l3-ba
landsat-c2l2alb-st
landsat-c2ard-sr
landsat-c2l2alb-sr
landsat-c2l2alb-ta
landsat-c2l3-dswe


In [41]:
for cltn in cltns:
    print(cltn,pd.read_csv(f'../../data/landsat/{cltn}_day_sum_LC09_LC08.csv').LC09.sum())

landsat-c2l2-sr 207126
landsat-c2l2-st 175629
landsat-c2ard-st 40887
landsat-c2l2alb-bt 13857
landsat-c2l3-fsca 0
landsat-c2ard-bt 41984
landsat-c2l1 239920
landsat-c2l3-ba 0
landsat-c2l2alb-st 13611
landsat-c2ard-sr 41984
landsat-c2l2alb-sr 13857
landsat-c2l2alb-ta 13857
landsat-c2l3-dswe 0
landsat-c2ard-ta 41984


## compare file entries on local node
First run `find . -type f -name '*.tar' -exec basename {} .tar \; > landsat_avail.txt` on local inventory

In [ ]:
glad21 = pd.read_csv("../../../data/Landsat/landsat9_avail_2021.txt",names=['scene_id'], header=None)

In [ ]:
glad22 = pd.read_csv("../../../data/Landsat/landsat_avail_2022.txt",names=['scene_id'], header=None)

In [ ]:
glad_inventory = pd.concat([glad21,glad22[glad22.scene_id.str.startswith("LC09")]])

In [ ]:
import glob

In [252]:
usgs_inventory = pd.concat([pd.read_csv(f) for f in glob.glob('../../data/stac/landsat-c2l1/csv/*.csv')])

In [100]:
set_finished = set(finished)

In [250]:
usgs_inventory

,id,properties.landsat:scene_id,properties.landsat:wrs_row,properties.landsat:wrs_type,properties.created,properties.landsat:cloud_cover_land,properties.view:sun_azimuth,properties.view:off_nadir,properties.platform,properties.landsat:wrs_path,...,properties.proj:transform,properties.eo:cloud_cover,properties.updated,properties.landsat:correction,properties.landsat:collection_category,properties.accuracy:geometric_y_bias,properties.accuracy:geometric_x_bias,properties.accuracy:geometric_rmse,properties.accuracy:geometric_x_stddev,properties.accuracy:geometric_y_stddev
0,LC09_L1GT_086110_20220306_20220307_02_T2,LC90861102022065LGN00,110,2,2022-07-06T21:44:09.129Z,0.02,57.163599,0.0,LANDSAT_9,86,...,"[30, 0, 1364085, 0, -30, -1346385]",0.02,2022-07-06T21:44:09.129Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
1,LC09_L1GT_086109_20220306_20220307_02_T2,LC90861092022065LGN00,109,2,2022-07-06T19:35:43.626Z,0.00,55.701679,0.0,LANDSAT_9,86,...,"[30, 0, 1415085, 0, -30, -1500885]",0.00,2022-07-06T19:35:43.626Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
2,LC09_L1GT_086108_20220306_20220307_02_T2,LC90861082022065LGN00,108,2,2022-07-06T19:43:04.674Z,0.00,54.467749,0.0,LANDSAT_9,86,...,"[30, 0, 1465785, 0, -30, -1656585]",0.00,2022-07-06T19:43:04.674Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
3,LC09_L1GT_086107_20220306_20220307_02_T2,LC90861072022065LGN00,107,2,2022-07-06T20:00:30.093Z,0.06,53.429951,0.0,LANDSAT_9,86,...,"[30, 0, 1515585, 0, -30, -1812885]",0.04,2022-07-06T20:00:30.093Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
4,LC09_L1GT_086106_20220306_20220307_02_T2,LC90861062022065LGN00,106,2,2022-07-06T20:26:01.021Z,-1.00,52.560728,0.0,LANDSAT_9,86,...,"[30, 0, 1564785, 0, -30, -1970385]",99.30,2022-07-06T20:26:01.021Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,LO09_L1TP_094062_20220315_20220315_02_T1,LO90940622022074LGN00,62,2,2022-07-01T14:13:35.912Z,19.15,89.420927,0.0,LANDSAT_9,94,...,"[30, 0, 176685, 0, -30, -204285]",19.46,2022-07-01T14:13:35.912Z,L1TP,T1,0.0,0.0,6.847,4.645,5.030
639,LO09_L1GT_094061_20220315_20220315_02_T2,LO90940612022074LGN00,61,2,2022-07-06T19:31:12.508Z,1.34,91.978171,0.0,LANDSAT_9,94,...,"[30, 0, 210585, 0, -30, -44385]",58.57,2022-07-06T19:31:12.508Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
640,LO09_L1TP_094056_20220315_20220315_02_T1,LO90940562022074LGN00,56,2,2022-07-01T11:00:32.252Z,9.96,105.035320,0.0,LANDSAT_9,94,...,"[30, 0, 381885, 0, -30, 755115]",6.76,2022-07-01T11:00:32.252Z,L1TP,T1,0.0,0.0,8.692,5.258,6.921
641,LO09_L1GT_094055_20220315_20220315_02_T2,LO90940552022074LGN00,55,2,2022-07-06T18:44:47.153Z,9.00,107.614333,0.0,LANDSAT_9,94,...,"[30, 0, 416085, 0, -30, 914715]",9.73,2022-07-06T18:44:47.153Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN


In [242]:
usgs_inventory['properties.platform']

0      LANDSAT_9
1      LANDSAT_9
2      LANDSAT_9
3      LANDSAT_9
4      LANDSAT_9
         ...    
638    LANDSAT_9
639    LANDSAT_9
640    LANDSAT_9
641    LANDSAT_9
642    LANDSAT_9
Name: properties.platform, Length: 115355, dtype: object

In [104]:
usgs_inventory.head()

,id,properties.landsat:scene_id,properties.landsat:wrs_row,properties.landsat:wrs_type,properties.created,properties.landsat:cloud_cover_land,properties.view:sun_azimuth,properties.view:off_nadir,properties.platform,properties.landsat:wrs_path,...,properties.proj:transform,properties.eo:cloud_cover,properties.updated,properties.landsat:correction,properties.landsat:collection_category,properties.accuracy:geometric_y_bias,properties.accuracy:geometric_x_bias,properties.accuracy:geometric_rmse,properties.accuracy:geometric_x_stddev,properties.accuracy:geometric_y_stddev
0,LC09_L1GT_086110_20220306_20220307_02_T2,LC90861102022065LGN00,110,2,2022-07-06T21:44:09.129Z,0.02,57.163599,0.0,LANDSAT_9,86,...,"[30, 0, 1364085, 0, -30, -1346385]",0.02,2022-07-06T21:44:09.129Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
1,LC09_L1GT_086109_20220306_20220307_02_T2,LC90861092022065LGN00,109,2,2022-07-06T19:35:43.626Z,0.00,55.701679,0.0,LANDSAT_9,86,...,"[30, 0, 1415085, 0, -30, -1500885]",0.00,2022-07-06T19:35:43.626Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
2,LC09_L1GT_086108_20220306_20220307_02_T2,LC90861082022065LGN00,108,2,2022-07-06T19:43:04.674Z,0.00,54.467749,0.0,LANDSAT_9,86,...,"[30, 0, 1465785, 0, -30, -1656585]",0.00,2022-07-06T19:43:04.674Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
3,LC09_L1GT_086107_20220306_20220307_02_T2,LC90861072022065LGN00,107,2,2022-07-06T20:00:30.093Z,0.06,53.429951,0.0,LANDSAT_9,86,...,"[30, 0, 1515585, 0, -30, -1812885]",0.04,2022-07-06T20:00:30.093Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN
4,LC09_L1GT_086106_20220306_20220307_02_T2,LC90861062022065LGN00,106,2,2022-07-06T20:26:01.021Z,-1.00,52.560728,0.0,LANDSAT_9,86,...,"[30, 0, 1564785, 0, -30, -1970385]",99.30,2022-07-06T20:26:01.021Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN


In [254]:
usgs_inventory['aq_time'] = pd.to_datetime(usgs_inventory['properties.datetime'])

In [267]:
set(usgs_inventory.id.to_list())-set(finished_m2m)

set()

In [ ]:
set(usgs_inventory.id.to_list())-set(finished_m2m)

In [265]:
len(finished_m2m)

115355

In [266]:
usgs_inventory.shape

(115355, 27)

In [ ]:
m2m_batch1 = usgs_inventory[usgs_inventory['aq_time']<"2021-12-01"].id.to_list()

In [240]:
len(finished_m2m)

74430

In [188]:
set(m2m_batch2)-set(finished_m2m)

600

In [196]:
export_scenes_list(list(set(m2m_batch2)-set(finished_m2m)),'batch2',200)

Done 3


In [176]:
m2m_batch2 = usgs_inventory[("2021-11-30" < usgs_inventory['aq_time'] )&(usgs_inventory['aq_time']< "2022-01-01")].sort_values('aq_time').id.to_list()

In [208]:
batch3 = usgs_inventory[("2022-01-01" <= usgs_inventory['aq_time'] )&(usgs_inventory['aq_time']< "2022-02-01")].sort_values('aq_time').id.to_list()

In [228]:
set(batch3)-set(finished_m2m)

set()

In [212]:
export_scenes_list(list(set(batch3)-set(finished_m2m)),'batch3',200)

Done 2


In [213]:
batch4 = usgs_inventory[("2022-02-01" <= usgs_inventory['aq_time'] )&(usgs_inventory['aq_time']< "2022-03-01")].sort_values('aq_time').id.to_list()

In [214]:
usgs_inventorylen(batch4)

19368

In [235]:
batch4 = list(set(batch4)-set(finished_m2m))

In [236]:
export_scenes_list(batch4,'batch4',200)

Done 27


In [223]:
batch5 = usgs_inventory[("2022-03-01" <= usgs_inventory['aq_time'] )&(usgs_inventory['aq_time']< "2022-04-01")].sort_values('aq_time').id.to_list()

In [237]:
batch5 = list(set(batch5)-set(finished_m2m))

In [238]:
len(batch5)

14600

In [239]:
export_scenes_list(batch5,'batch5',200)

Done 73


In [255]:
batch6 = usgs_inventory[("2022-04-01" <= usgs_inventory['aq_time'] )&(usgs_inventory['aq_time']< "2022-05-01")].sort_values('aq_time').id.to_list()

In [257]:
len(batch6)

20925

In [259]:
export_scenes_list(batch6,'batch6',200)

Done 105


In [121]:
usgs_inventory[("2021-11-30" < usgs_inventory['aq_time'] )&(usgs_inventory['aq_time']< "2022-01-01")].sort_values('aq_time')

,id,properties.landsat:scene_id,properties.landsat:wrs_row,properties.landsat:wrs_type,properties.created,properties.landsat:cloud_cover_land,properties.view:sun_azimuth,properties.view:off_nadir,properties.platform,properties.landsat:wrs_path,...,properties.eo:cloud_cover,properties.updated,properties.landsat:correction,properties.landsat:collection_category,properties.accuracy:geometric_y_bias,properties.accuracy:geometric_x_bias,properties.accuracy:geometric_rmse,properties.accuracy:geometric_x_stddev,properties.accuracy:geometric_y_stddev,aq_time
119,LC09_L1GT_043018_20211203_20220123_02_T2,LC90430182021337LGN03,18,2,2022-07-06T20:58:05.979Z,100.00,170.770645,0.0,LANDSAT_9,43,...,100.00,2022-07-06T20:58:05.979Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN,2021-12-03 18:33:56.723690+00:00
118,LC09_L1TP_043019_20211203_20220123_02_T1,LC90430192021337LGN03,19,2,2022-07-01T10:19:42.440Z,85.73,169.999815,0.0,LANDSAT_9,43,...,85.73,2022-07-01T10:19:42.440Z,L1TP,T1,0.0,0.0,9.875,5.490,8.208,2021-12-03 18:34:20.601997+00:00
117,LC09_L1TP_043020_20211203_20220123_02_T1,LC90430202021337LGN03,20,2,2022-07-01T10:30:34.830Z,13.23,169.275627,0.0,LANDSAT_9,43,...,13.23,2022-07-01T10:30:34.830Z,L1TP,T1,0.0,0.0,9.573,5.381,7.917,2021-12-03 18:34:44.480305+00:00
116,LC09_L1TP_043021_20211203_20220123_02_T1,LC90430212021337LGN03,21,2,2022-07-01T14:00:36.378Z,19.28,168.590462,0.0,LANDSAT_9,43,...,19.28,2022-07-01T14:00:36.378Z,L1TP,T1,0.0,0.0,8.833,5.230,7.118,2021-12-03 18:35:08.358611+00:00
115,LC09_L1TP_043022_20211203_20220123_02_T1,LC90430222021337LGN03,22,2,2022-07-01T10:19:47.573Z,13.50,167.937341,0.0,LANDSAT_9,43,...,13.50,2022-07-01T10:19:47.573Z,L1TP,T1,0.0,0.0,9.504,6.050,7.329,2021-12-03 18:35:32.241156+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,LC09_L1GT_079120_20211231_20220121_02_T2,LC90791202021365LGN01,120,2,2022-07-06T21:10:29.961Z,0.00,101.638044,0.0,LANDSAT_9,79,...,0.00,2022-07-06T21:10:29.961Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN,2021-12-31 22:56:59.183926+00:00
3,LC09_L1GT_079121_20211231_20220121_02_T2,LC90791212021365LGN01,121,2,2022-07-06T20:02:47.521Z,0.00,110.731190,0.0,LANDSAT_9,79,...,0.00,2022-07-06T20:02:47.521Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN,2021-12-31 22:57:23.163897+00:00
2,LC09_L1GT_079122_20211231_20220121_02_T2,LC90791222021365LGN01,122,2,2022-07-06T19:37:13.889Z,0.00,120.417011,0.0,LANDSAT_9,79,...,0.00,2022-07-06T19:37:13.889Z,L1GT,T2,NaN,NaN,NaN,NaN,NaN,2021-12-31 22:57:47.148105+00:00
1,LC09_L1TP_095021_20211231_20220123_02_T1,LC90950212021365LGN01,21,2,2022-07-01T10:36:06.422Z,18.18,165.600071,0.0,LANDSAT_9,95,...,15.42,2022-07-01T10:36:06.422Z,L1TP,T1,0.0,0.0,9.672,7.081,6.588,2021-12-31 23:56:22.855068+00:00


In [112]:
usgs_inventory[pd.to_datetime(usgs_inventory['properties.datetime'])<"2021-12-01"

0      True
1      True
2      True
3      True
4      True
       ... 
638    True
639    True
640    True
641    True
642    True
Name: properties.datetime, Length: 115355, dtype: bool

In [89]:
set_usgs = set(usgs_inventory.id.tolist())

In [103]:
todo = list(set_usgs - set_finished)
todo

In [ ]:
inventory_merged = usgs_inventory.merge(glad_inventory,left_on='id',right_on='scene_id',how='left')

missing = inventory_merged.query('scene_id.isnull()',engine='python')

missing.shape

missing[['id']].to_csv('../../../data/Landsat/missing_id.csv',index=False)
missing.dtypes

### Spatial and temporal visualization on the missing scenes

In [ ]:
import seaborn as sns

In [ ]:
pathrow_missing = missing.groupby(["properties.landsat:wrs_path","properties.landsat:wrs_row"]).agg({'id':'count'}).reset_index().set_axis(['path','row','count'],axis=1)\
    .pivot(index='row',columns='path',values='count').fillna(0)

In [ ]:
sns.heatmap(pathrow_missing,cbar_kws={'label': 'missing scenes'})


In [ ]:
missing['properties.datetime'] = missing['properties.datetime'].astype('datetime64[ns]')

In [ ]:
sns.displot(data=missing,x='properties.datetime',aspect=4,bins=360)